# Training Model 

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
train = pd.read_csv("data/feature_engineering/loans_train.csv")
test = pd.read_csv("data/feature_engineering/loans_test.csv")
valid = pd.read_csv("data/feature_engineering/loans_valid.csv")

# Training Model Preparation

In [53]:
# Use Cleaned Datasets for Model Training

from signal import valid_signals
from sklearn.impute import SimpleImputer

# Use your cleaned datasets that already have datetime features extracted
train = train[train['target'] == 0]
X_train = train.drop(columns=['index', 'target'], errors='ignore')
y_train = train['target']
X_valid = valid.drop(columns=['index', 'target'], errors='ignore')
y_valid = valid['target']
X_test = test.drop(columns=['Id'], errors='ignore')

# Ensure all datasets have the same columns
common_cols = set(X_train.columns) & set(X_valid.columns) & set(X_test.columns)
X_train = X_train[list(common_cols)]
X_valid = X_valid[list(common_cols)]
X_test = X_test[list(common_cols)]

# Drop all string/categorical columns (union across splits)
obj_cols = set(X_train.select_dtypes(include=['object', 'category']).columns)
obj_cols |= set(X_valid.select_dtypes(include=['object', 'category']).columns)
obj_cols |= set(X_test.select_dtypes(include=['object', 'category']).columns)
obj_cols = list(obj_cols)

X_train = X_train.drop(columns=obj_cols, errors='ignore')
X_valid = X_valid.drop(columns=obj_cols, errors='ignore')
X_test  = X_test.drop(columns=obj_cols, errors='ignore')


# Fill remaining NaN values using robust methods
print("Step 2: Fill remaining NaN values")
print(f"NaN count in X_train: {X_train.isna().sum().sum()}")
print(f"NaN count in X_valid: {X_valid.isna().sum().sum()}")
print(f"NaN count in X_test: {X_test.isna().sum().sum()}")

# For numeric columns, use median imputation
# 1. 找出所有数值类型的列
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
print(f"Found {len(numeric_cols)} numeric columns for imputation.")
# 2. 遍历每一列进行中位数填充
for col in numeric_cols:
    # 检查训练集中是否有 NaN
    if X_train[col].isna().any():
        # 2a. 仅从训练集中计算中位数
        median_val = X_train[col].median()
        # 2b. 处理边缘情况：如果训练集的该列全是 NaN，则中位数也是 NaN
        if pd.isna(median_val):
            median_val = 0  # 默认填充为 0
        # 2c. 使用计算出的中位数填充所有数据集
        X_train[col] = X_train[col].fillna(median_val)
        X_valid[col] = X_valid[col].fillna(median_val)
        X_test[col] = X_test[col].fillna(median_val)
print("Median imputation complete.")

# For categorical columns, use mode imputation
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns
for col in categorical_cols:
    if X_train[col].isna().any():
        mode_val = X_train[col].mode()
        if len(mode_val) > 0:
            X_train[col] = X_train[col].fillna(mode_val[0])
            X_valid[col] = X_valid[col].fillna(mode_val[0])
            X_test[col] = X_test[col].fillna(mode_val[0])
        else:
            # If no mode exists, use 'unknown'
            X_train[col] = X_train[col].fillna('unknown')
            X_valid[col] = X_valid[col].fillna('unknown')
            X_test[col] = X_test[col].fillna('unknown')

# 3. Verify the fix results
print("Step 3: Verify fix results")
print(f"NaN count in X_train: {X_train.isna().sum().sum()}")
print(f"NaN count in X_valid: {X_valid.isna().sum().sum()}")
print(f"NaN count in X_test: {X_test.isna().sum().sum()}")


Step 2: Fill remaining NaN values
NaN count in X_train: 52690
NaN count in X_valid: 9584
NaN count in X_test: 22635
Found 298 numeric columns for imputation.
Median imputation complete.
Step 3: Verify fix results
NaN count in X_train: 0
NaN count in X_valid: 0
NaN count in X_test: 0


# Feature Selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import average_precision_score, roc_auc_score

# Ensure consistent columns before scaling (important!)
common_cols = list(set(X_train.columns) & set(X_valid.columns))
X_train = X_train[common_cols]
X_valid = X_valid[common_cols]
X_test = X_test[common_cols]

# Drop non-numeric columns AFTER identifying common columns
object_cols = X_train.select_dtypes(include=['object']).columns
X_train = X_train.drop(columns=object_cols)
X_valid = X_valid.drop(columns=object_cols)
X_test = X_test.drop(columns=object_cols)

# Store feature names BEFORE scaling
feature_names = X_train.columns.tolist()

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)


print("🚀 Executing Feature Selection...")
print("   (Using RandomForestClassifier trained on the validation set to find important features...)")

# Define the selector model
rf_selector = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Train the selector on the VALIDATION data (X_valid_scaled, y_valid)
# This uses the labels (0s and 1s) in the validation set to determine feature importance
rf_selector.fit(X_valid_scaled, y_valid)

# Get feature importances
importances = rf_selector.feature_importances_

# Check if lengths match (should match if scaler and fit used correctly)
if len(feature_names) == len(importances):
    # Create importance DataFrame
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values(by='importance', ascending=False)

    # Select Top N features
    TOP_N_FEATURES = 75 # You can adjust this number
    top_features = feature_importance_df.head(TOP_N_FEATURES)['feature'].tolist()

    print(f"✅ Selected Top {TOP_N_FEATURES} features.")
    print("Top 5 most important features:", top_features[:5])

    # Create new DataFrames/arrays containing only the selected features
    # Need to convert scaled arrays back to DataFrame temporarily to select by name
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_names)
    X_valid_scaled_df = pd.DataFrame(X_valid_scaled, columns=feature_names)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)

    # final selected features df
    X_train_selected = X_train_scaled_df[top_features]
    X_valid_selected = X_valid_scaled_df[top_features]
    X_test_selected = X_test_scaled_df[top_features]


    # Create the 'normal only' training set for unsupervised models
    # Convert y_train to numpy array if it's a pandas Series for boolean indexing
    y_train_np = y_train.to_numpy() if isinstance(y_train, pd.Series) else y_train
    X_train_normal_selected = X_train_selected[y_train_np == 0]

    print(f"   → Created X_train_selected with shape: {X_train_selected.shape}")
    print(f"   → Created X_valid_selected with shape: {X_valid_selected.shape}")
    print(f"   → Created X_test_selected with shape: {X_test_selected.shape}")
    print(f"   → Created X_train_normal_selected (for training) with shape: {X_train_normal_selected.shape}")

else:
    print("--- ERROR ---")
    print("Length mismatch between feature names and importances.")
    print(f"Length of feature_names (from X_train before scaling): {len(feature_names)}")
    print(f"Length of importances (from RandomForest): {len(importances)}")
    print("This indicates an issue in data preparation before scaling or in the selector training.")
    print("Please check the steps before this cell.")

🚀 Executing Feature Selection...
   (Using RandomForestClassifier trained on the validation set to find important features...)
✅ Selected Top 75 features.
Top 5 most important features: ['InterestBearingUPB_num_decrease', 'CurrentActualUPB_late_slope', 'InterestBearingUPB_late_std', 'CurrentActualUPB_num_decrease', 'CreditScore']
   → Created X_train_selected with shape: (30504, 75)
   → Created X_valid_selected with shape: (5370, 75)
   → Created X_test_selected with shape: (13426, 75)
   → Created X_train_normal_selected (for training) with shape: (30504, 75)


# Baseline Model

In [55]:
# Print target=1 ratio in valid (concise)
print(f"valid positive ratio: {y_valid.mean():.4f}")

valid positive ratio: 0.1261


In [56]:
# =============================================================
# Baseline Model Evaluation (Using TOP N Selected Features - CORRECTED)
# =============================================================
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM
from sklearn.metrics import average_precision_score, roc_auc_score
import numpy as np
import pandas as pd
# No StandardScaler needed here as selected features are already scaled

# Check if feature selection variables exist
if 'TOP_N_FEATURES' not in locals() or \
   'X_train_selected' not in locals() or \
   'X_valid_selected' not in locals() or \
   'y_train' not in locals() or \
   'y_valid' not in locals():
    print("--- ERROR ---")
    print("Required variables (TOP_N_FEATURES, X_train_selected, X_valid_selected, y_train, y_valid) not found.")
    print("Please ensure you have successfully run the 'Feature Selection' cell (id: d5b93fe1) immediately before this cell.")
else:
    print(f"🚀 Running Baseline Models (Using Top {TOP_N_FEATURES} Selected & Scaled Features)...")

    # ---------- Data Preparation ----------
    # Use variables from feature selection cell (already scaled)
    X_train_ = X_train_selected.copy()
    X_valid_ = X_valid_selected.copy()
    y_train_ = y_train.copy() # Should contain only 0s
    y_valid_ = y_valid.copy() # Contains 0s and 1s

    # (===== ⭐️ Crucial Step: Create the 'normal only' subset for fitting =====)
    X_train_fit_normal = X_train_[y_train_ == 0]

    print(f"   → Using {X_train_fit_normal.shape[0]} normal training samples for fitting (with {X_train_fit_normal.shape[1]} features).")
    print(f"   → Evaluating on {X_valid_.shape[0]} validation samples (with {X_valid_.shape[1]} features).")

    # ---------- Define Models ----------
    # Hyperparameters might need re-tuning based on the selected features
    models = {
        "IsolationForest": IsolationForest(
            n_estimators=200,
            contamination=0.05, # Hyperparameter to tune
            random_state=42
        ),
        "LocalOutlierFactor": LocalOutlierFactor(
            n_neighbors=10,       # Hyperparameter to tune
            novelty=True,
            contamination=0.05  # Hyperparameter to tune
        ),
        "PCA_Reconstruction": PCA(
            n_components=0.95,
            random_state=42     # n_components might need tuning
        ),
        "OneClassSVM": OneClassSVM(
            kernel="rbf",
            gamma="scale",
            nu=0.05             # Hyperparameter to tune
        )
    }

    # ---------- Evaluation Loop ----------
    results_selected = [] # Use a distinct list name

    for name, model in models.items():
        print(f"    Fitting {name}...")
        scores = None
        try:
            # (===== ⭐️ Corrected Fit Call =====)
            # Fit ALL models ONLY on normal samples
            model.fit(X_train_fit_normal)

            # (===== ⭐️ Corrected Evaluation Call =====)
            # Evaluate ALL models on the full validation set (X_valid_)
            if name == "PCA_Reconstruction":
                # Ensure X_valid_ is NumPy for calculation if it's a DataFrame
                X_valid_np = X_valid_.values if isinstance(X_valid_, pd.DataFrame) else X_valid_
                Xv_rec = model.inverse_transform(model.transform(X_valid_np))
                scores = np.mean((X_valid_np - Xv_rec) ** 2, axis=1) # Higher error = more anomalous
            else: # IF, LOF, OCSVM
                # Ensure X_valid_ is compatible with decision_function (DataFrame is usually fine)
                scores = -model.decision_function(X_valid_) # Higher score = more anomalous

            # --- Calculate Metrics ---
            if scores is None or np.any(np.isnan(scores)) or np.any(np.isinf(scores)):
                 valid_scores = scores[~(np.isnan(scores) | np.isinf(scores))] if scores is not None else np.array([0.0])
                 median_score = np.median(valid_scores) if len(valid_scores) > 0 else 0.0
                 scores = np.nan_to_num(scores, nan=median_score, posinf=median_score, neginf=median_score)

            if np.sum(y_valid_) == 0 or np.all(y_valid_ == y_valid_[0]):
                 ap, roc = np.nan, np.nan
            else:
                 ap = average_precision_score(y_valid_, scores)
                 roc = roc_auc_score(y_valid_, scores)

        except Exception as e:
            print(f"     ERROR fitting or predicting {name}: {e}")
            ap, roc = np.nan, np.nan

        results_selected.append((name, round(ap, 4), round(roc, 4)))
        print(f"     {name} -> AP: {ap:.4f}, ROC-AUC: {roc:.4f}")

    # ---------- Output Results ----------
    results_df_selected = pd.DataFrame(results_selected, columns=["Model", "AP", "ROC-AUC"]).sort_values("AP", ascending=False)
    print(f"\n✅ Baseline Results (Top {X_train_fit_normal.shape[1]} Scaled Features, Trained on Normal Samples Only):")
    print(results_df_selected)

🚀 Running Baseline Models (Using Top 75 Selected & Scaled Features)...
   → Using 30504 normal training samples for fitting (with 75 features).
   → Evaluating on 5370 validation samples (with 75 features).
    Fitting IsolationForest...
     IsolationForest -> AP: 0.1414, ROC-AUC: 0.5779
    Fitting LocalOutlierFactor...


/Users/sixestates/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LocalOutlierFactor was fitted with feature names
  warnings.warn(


     LocalOutlierFactor -> AP: 0.2803, ROC-AUC: 0.6743
    Fitting PCA_Reconstruction...
     PCA_Reconstruction -> AP: 0.1868, ROC-AUC: 0.5884
    Fitting OneClassSVM...


/Users/sixestates/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(


     OneClassSVM -> AP: 0.2061, ROC-AUC: 0.6497

✅ Baseline Results (Top 75 Scaled Features, Trained on Normal Samples Only):
                Model      AP  ROC-AUC
1  LocalOutlierFactor  0.2803   0.6743
3         OneClassSVM  0.2061   0.6497
2  PCA_Reconstruction  0.1868   0.5884
0     IsolationForest  0.1414   0.5779


# Model Training